In [3]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will use a toy dataset from sklearn.

In [4]:
from sklearn import datasets

#Load dataset
sklearn_dataset = datasets.load_breast_cancer()
Xx  = sklearn_dataset.data
y = sklearn_dataset.target * 2 - 1    # labels must be in {-1, 1} for the hinge loss
X = np.ones((Xx.shape[0], Xx.shape[1] + 1 ))    # add a column of ones for intercept
X[:, :-1] = Xx
print("(N, D) =", X.shape)

(N, D) = (569, 31)


## Prepare cost and prediction functions

The primal objective for SVM is:

$$\mathcal L (\mathbf w) = \sum_{n=1}^N [1- y_n \mathbf x_n^\top \mathbf w]_+ +  \frac{\lambda}{2}\mathbf w^\top \mathbf w$$

In [5]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    loss = np.maximum(0, (1 - y * X.dot(w))).sum()
    reg = lambda_ / 2 * w.dot(w)
    return loss + reg

In [6]:
xtest = np.array([[0,1,-1]])
wtest = np.array([-2,0,1])
ytest = np.array([1])
assert calculate_primal_objective(ytest, xtest, wtest, 0) == 2

In [7]:
def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    yhat = X.dot(w) >= 0
    # Map to {-1, 1}
    yhat = (yhat * 2) - 1
    return (yhat == y).mean()

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

---

Given

$$l:\mathbb R \rightarrow \mathbb R, \; z \mapsto \max(0, 1-z) = 
\begin{cases}
1 - z & \text{if } z \le 1\\
0 & \text{otherwise}
\end{cases}$$

We have

$$\frac{\partial l}{\partial z} (z) = \begin{cases}
-1 & \text{if } z < 1\\
0&\text{otherwise}
\end{cases}$$

and is not defined for $z=1$. We thus have to compute the **subgradient**:

$$\partial l(z=1) \in [-1; 0]$$

We could theoretically pick any value. When $z=1$, we've just reached a value we are confident enough in so that we do not penalize it anymore. We might want a non-zero subgradient value for $z=1$ so that we have a bit more margin of confidence. Some value between -1 and 0 (e.g. -0.5) would probably be the best (?), but if we care about implementation efficiency, we should consider either -1 or 0.

Let's call $D_{l(z)}$ the gradient of $l$ where it is defined and $-1$ when it's not. Using the chain rule of derivation:

$$D_{l(\mathbf z = y_n\mathbf x_n^\top\mathbf w)} = \begin{cases}
-y_n\mathbf x_n & \text{if } y_n\mathbf x_n^\top\mathbf w \le 1\\
0 &\text{otherwise}
\end{cases}$$

---

In [8]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    x_n, y_n = X[n], y[n]
    
    # Save element-wise product to perform it only once
    ynxn = y_n * x_n
    non_zero_case = ynxn.dot(w) <= 1
    
    return non_zero_case * (-ynxn)

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [9]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 2 * int(1e5)
    gamma = 1e-4
    lambda_ = int(1e4)   # big because scales with N due to the formulation of the problem (not an averaged loss)
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}\t\tcost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0		cost=27066.623134253427
iteration=10000		cost=189.15199743233316
iteration=20000		cost=184.66280647437662
iteration=30000		cost=181.10636971056587
iteration=40000		cost=179.37656683298607
iteration=50000		cost=178.14760046011654
iteration=60000		cost=177.95644273859145
iteration=70000		cost=176.26779852168
iteration=80000		cost=175.56857472672067
iteration=90000		cost=175.2921848382444
iteration=100000		cost=174.50550104168371
iteration=110000		cost=174.21786089935196
iteration=120000		cost=173.60987458774687
iteration=130000		cost=173.25976713469763
iteration=140000		cost=172.916035326084
iteration=150000		cost=172.69622051738588
iteration=160000		cost=172.3507235687213
iteration=170000		cost=172.03932105643509
iteration=180000		cost=171.7956805766345
iteration=190000		cost=171.5590955343336
training accuracy = 0.9050966608084359


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

---

**Dual optimization problem**: 

$$f(\mathbf \alpha) = \mathbf \alpha^\top \mathbf 1 - \frac{1}{2\lambda} YXX^\top Y \alpha, \quad 0 \le \alpha_n \le 1 \; \forall n$$

where $Y := \text{diag}(\mathbf y)$ and $X\in \mathbb R^{N\times D}$. The function $f$ is related to the loss via the formula:

$$\min_{\mathbf w} \mathcal L (\mathbf w) = \max_{\mathbf \alpha \in \mathbb [0;1]^N} f (\mathbf \alpha)$$

We first optimize over the dual variable $\mathbf \alpha$ and then map the solution back to $\mathbf w$:

$$\begin{align}
\mathbf \alpha^\star &= \arg\max_{\mathbf \alpha \in \mathbb [0;1]^N} f(\mathbf\alpha)\\
\mathbf w^\star &= X^\top Y \mathbf \alpha^\star
\end{align}$$

We want to perform **coordinate descent** for SVM. For every iteration, we pick a coordinate $n\in[N]$ uniformly at random and maximize the objective 

$$\max_{\gamma\in\mathbb R} f(\mathbf\alpha + \gamma \mathbf e_n) \quad \text{such that } 0 \le \alpha_n+\gamma \le 1$$

The derivation is pretty cumbersome to report here, see the solutions of lab 7 instead :).

---

In [35]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    
    gamma = lambda_ / x_n.dot(x_n) * (1 - y_n * x_n.T.dot(w))
    
    alpha_n = old_alpha_n + gamma
    # Constraint: project alpha_n to [0, 1]
    alpha_n = np.clip(alpha_n, 0, 1)
    alpha[n] = alpha_n
    
    
    
    return w, alpha

Since we have

$$\mathbf w = \frac 1 \lambda X^\top Y \mathbf \alpha$$

we can rewrite a part of the dual objective:

$$\begin{align}
\frac 1 {2\lambda} \mathbf \alpha^\top YXX^\top Y \mathbf \alpha 
&= \frac 1 {2\lambda} \left(\mathbf \alpha^\top YX\right) \left(X^\top Y \mathbf \alpha \right)\\
&= \frac 1 {2\lambda} \left(X^\top Y \mathbf \alpha \right)^\top \left(X^\top Y \mathbf \alpha \right)\\
&= \frac \lambda 2 \mathbf w ^\top \mathbf w
\end{align}$$

so that

$$f(\mathbf \alpha) = \mathbf \alpha ^\top \mathbf 1 - \frac \lambda 2 \mathbf w^\top \mathbf w$$

In [31]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    return alpha.sum() - lambda_ / 2.0 * w.T.dot(w)

In [36]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 2*int(1e5)
    lambda_ = int(1e4)   # use same lambda as before in order to compare

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:569.00000, dual:0.01244, gap:568.98756
iteration=10000, primal:569.00000, dual:162.91750, gap:406.08250
iteration=20000, primal:569.00000, dual:279.24576, gap:289.75424
iteration=30000, primal:569.00000, dual:354.12922, gap:214.87078
iteration=40000, primal:569.00000, dual:401.11057, gap:167.88943
iteration=50000, primal:569.00000, dual:432.09847, gap:136.90153
iteration=60000, primal:569.00000, dual:453.38778, gap:115.61222
iteration=70000, primal:569.00000, dual:468.38368, gap:100.61632
iteration=80000, primal:569.00000, dual:479.80254, gap:89.19746
iteration=90000, primal:569.00000, dual:488.63511, gap:80.36489
iteration=100000, primal:569.00000, dual:496.48992, gap:72.51008
iteration=110000, primal:569.00000, dual:503.25589, gap:65.74411
iteration=120000, primal:569.00000, dual:509.22739, gap:59.77261
iteration=130000, primal:569.00000, dual:514.88494, gap:54.11506
iteration=140000, primal:569.00000, dual:520.01306, gap:48.98694
iteration=150000, primal:569.0000